In [ ]:
#########################################################
#                                                       #
#                      homca v.1.0                      #
#                                                       #
# Set up homca below by sepcifying the directory and    #
# name of the output file, and choose what parts of the #
# result you would like to print.                       #
#                                                       #
# © Damian Iltgen, 2021                                 #
#                                                       #
#########################################################

import os
from copy import deepcopy

#########################################################
#                                                       #
#                         Setup                         #
#                                                       #
#########################################################

# Specify directory that contains the output file (e.g. the path of khoca)
# and the name of the output file (needs to be a plain text file)
os.chdir("/path/to/your/output/file")
fname = "filename.txt"

# Define what to print

# Print reduced matrices [bool (print), bool (print in LaTeX)]
prntmatrices = [True, False]

# Print elementary pieces [bool (print), bool (print matrix), bool (print quantum degree)]
prntpieces = [False, False, False]

# Print summands [bool (print), bool (print matrix), bool (print quantum degree),
# str (sorted by "rank" or "length"), str (ordered "asc" or "desc"),
# bool (hide summands of length < 3)]
prntchains = [True, True, False, "rank", "asc", False]

# Print piece of odd rank [bool (print), bool (print matrix), bool (print quantum degree)]
prntoddpiece = [True, False, True]

# NOTE: In the source code, direct summands are referred to
# as "chains". In particular, "chains" as used below are not
# to be confused with the mathematical notion of "chain"

#########################################################
#                                                       #
#      End of Setup, modify code below at own risk!     #
#                                                       #
#########################################################

######################## Classes ########################

# Base class for subcomplexes. When computing the Z[G]-complex with khoca, some of the
# chain objects can be zero leading to several subcomplexes. A subcomplex contains
# a list differentials, a list of matrices (one for each differential), a list of
# the involved homological and quantum degrees, as well as a list of chains.
class Subcomplex:
    def __init__(self):
        self.scMatrices = []
        self.scHdegs = []
        self.scQdegs = []
        self.scDifferentials = []
        self.scChains = []

# Base class for differentials. Stores the homological degree, the matrix, quantum degrees
# of the domain and the range, a list of elementary pieces ("pieces"), and a list of pieces
# ("temppieces") that are used to construct the elementary ones.
class Differential:
    def __init__(self, hdeg, matrix, qdegd, qdegr):
        self.hdeg = hdeg
        self.matrix = matrix
        self.qdegd = qdegd
        self.qdegr = qdegr
        self.epieces = []
        self.temppieces = []

# A class for chains. A chain is a sequence of one or more Pieces such that the list of targets
# of every piece has non-empty intersection with the list of sources of the following
# piece in the sequence. A Chain stores the chain as a list together with a list of differentials
# corresponding to the pieces in the chain. It also stores the homological degree of the start and
# the end of the chain, as well as its rank and total length.
class Chain:
    def __init__(self, chain, differentials, start, stop):
        self.cChain = chain
        self.cDiffs = differentials
        self.cStart = start
        self.cStop = stop
        self.cLength = self.cStop - self.cStart + 1
        self.cRank = 0
        self.cMatrices = []
        
        # Construct the chain by extending sources resp. targets of
        # consecutive pieces and construct the matrix.
        #
        # REWORK, MIGHT NO LONGER BE NECESSARY
        #
        for i in range(len(self.cChain)-1):
            if len(self.cChain[i].pTargets) < len(self.cChain[i+1].pSources):
                self.cChain[i].extendtargets(self.cChain[i+1])
            elif len(self.cChain[i].pTargets) > len(self.cChain[i+1].pSources):
                self.cChain[i+1].extendsources(self.cChain[i])
            elif self.cChain[i].pTargets != self.cChain[i+1].pSources:
                self.cChain[i].extendtargets(self.cChain[i+1])
                self.cChain[i+1].extendsources(self.cChain[i])
        
        # Compute the rank of the chain
        self.computerank()
    
    # Computes the rank of the chain
    def computerank(self):
        if len(self.cChain) == 1:
            if self.cChain[0].haszeromap() == True:
                self.cRank = len(self.cChain[0].pSources)
            else:
                self.cRank = len(self.cChain[0].pSources) + len(self.cChain[0].pTargets)
        if len(self.cChain) > 1:
            self.cRank = len(self.cChain[0].pSources) + len(self.cChain[0].pTargets)
            for i in range(1, len(self.cChain)):
                self.cRank += len(self.cChain[i].pTargets)
        
    # Print it pretty(,) please!
    def printit(self, printmatrix, printqdeg):
        cstr = ""
        for c in self.cChain:
            ind = self.cChain.index(c)
            hzm = c.haszeromap()
            if ind == 0 and hzm == True:
                cstr = cstr + c.sourcestring(self.cDiffs[ind], printqdeg)
            elif ind == 0 and hzm == False:
                cstr = (cstr + c.sourcestring(self.cDiffs[ind], printqdeg) + "\longrightarrow "
                        + c.targetstring(self.cDiffs[ind], printqdeg))
            else:
                cstr = cstr + "\longrightarrow " + c.targetstring(self.cDiffs[ind], printqdeg)
        # Print it!
        print("\nhdeg start: " + str(self.cStart) + ", hdeg stop: " + str(self.cStop) + ", length: "
              + str(self.cLength))
        pretty_print(LatexExpr(cstr))
        if printmatrix == True and hzm == False:
            for c in self.cChain:
                pretty_print(Matrix(R, c.pMatrix))

# Base class for maps. Here, a map has a single source, a single target, and a value.
# This class is needed to keep track of the source and target when building the matrix
# of a TargetGluedPiece.
class Map:
    def __init__(self, source, target, value):
        self.mSource = source
        self.mTarget = target
        self.mValue = value

# Main class for Pieces. A piece is a part of the transformation matrix of a differential and therefore
# has a number of sources and targets (that is, a domain and range) which are both ordered
# numerically and ascending. It also has a matrix that gets constructed from a 2d-nested list of Maps,
# which is sorted according to the sources (i.e. the i-th entry in pMaps corresponds to the i-th entry
# in pSources). (Note: an entry in pMaps can be thought of as the row vector representing the linear map from
# its corresponding source to some targets in pTargets.) The class also contains functions for merging
# two pieces and extending the sources resp. targets with those from another piece.
class Piece:
    def __init__(self, sources, targets, maps):
        self.pSources = sources
        self.pTargets = targets
        self.pMaps = maps
        self.pMatrix = []

    # Merges self with another piece
    def merge(self, piece):
        for s in piece.pSources:
            self.pSources.append(s)
            self.pSources.sort()
            self.pMaps.insert(self.pSources.index(s), piece.pMaps[piece.pSources.index(s)])
        self.pTargets = list(set(self.pTargets + piece.pTargets))
        self.pTargets.sort()
        self.constructmatrix()
        
    # Constructs the matrix of self. It is of the form [[row1],[row2],...,[rown]].
    def constructmatrix(self):
        
        temp1 = [] # Placeholder for creating a row
        temp2 = [] # Placeholder for storing rows
        
        # Every target space contributes a row, and the size
        # of a row is given by the number of sources.
        for j in self.pTargets:
            for i in range(len(self.pSources)):
                count = 0
                # Check if pSources[i] has a map with target j,
                # if not append 0 to the row (uses that pMaps is
                # ordered with respect to pSources).
                for m in self.pMaps[i]:
                    if j == m.mTarget:
                        temp1.append(m.mValue)
                        count += 1
                if count == 0:
                    temp1.append(0)
            temp2.append(temp1)
            temp1 = []
        self.pMatrix = temp2
    
    # Extend the sources of self with those of another piece
    def extendsources(self, piece):
        if self.haszeromap() == True or piece.haszeromap() == True:
            raise Exception("Error, a pawn is involved in source extension process!")
        sourcestoadd = [s for s in piece.pTargets if s not in self.pSources]
        if sourcestoadd == []:
            return False
        for s in sourcestoadd:
            self.pSources.append(s)
            self.pSources.sort()
            temp = []
            for t in self.pTargets:
                temp.append(Map(s,t,0))
            self.pMaps.insert(self.pSources.index(s), temp)
        self.constructmatrix()
        return True
    
    # Extend the targets of self with those of another piece
    def extendtargets(self, piece):
        if self.haszeromap() == True or piece.haszeromap() == True:
            raise Exception("Error a pawn is involved in target extension process!")
        targetstoadd = [t for t in piece.pSources if t not in self.pTargets]
        if targetstoadd == []:
            return False
        self.pTargets = list(set(self.pTargets + piece.pSources))
        self.pTargets.sort()
        self.constructmatrix()
        return True
    
    # Searches for elementary pieces within the specified differential (usually the one
    # self belongs to) that have common targets with self and merges them.
    def searchcommontargets(self, diff):
        i = 0
        while i < len(diff.epieces):
            if diff.epieces[i] == self or diff.epieces[i].haszeromap() == True:
                i += 1
                continue
            else:
                check = [t for t in diff.epieces[i].pTargets if t in self.pTargets]
                if check != []:
                    self.merge(diff.epieces[i])
                    diff.epieces.pop(i)
                    continue
                else:
                    i += 1
    
    # Searches for elementary pieces within the specified differential (usually the one
    # self belongs to) that have common sources with self and merges them.
    # (Currently nowhere used).
    def searchcommonsources(self, diff):
        i = 0
        while i < len(diff.epieces):
            if diff.epieces[i] == self or diff.epieces[i].haszeromap() == True:
                i += 1
                continue
            else:
                check = [s for s in diff.epieces[i].pSources if s in self.pSources]
                if check != []:
                    self.merge(diff.epieces[i])
                    diff.epieces.pop(i)
                    continue
                else:
                    i += 1
    
    # Checks if self has trivial map
    def haszeromap(self):
        if set([m.mValue for l in self.pMaps for m in l]) == {0}:
            return True
        else:
            return False
    
    # Constructs the domain as a string
    def sourcestring(self, diff, printqdeg):
        sstr = ""
        if printqdeg == True:
            # Construct the domain with qdegs
            for s in self.pSources:
                if self.pSources.index(s) < len(self.pSources)-1:
                    sstr = sstr + "R_{" + str(s) + "}\{" + str(diff.qdegd[s-1]) + "\}\oplus "
                else:
                    sstr = sstr + "R_{" + str(s) + "}\{" + str(diff.qdegd[s-1]) + "\}"
        else:
            # Construct the domain without qdegs
            for s in self.pSources:
                if self.pSources.index(s) < len(self.pSources)-1:
                    sstr = sstr + "R_{" + str(s) + "}\oplus "
                else:
                    sstr = sstr + "R_{" + str(s) + "}"
        return sstr
    
    # Constructs the taret space as a string
    def targetstring(self, diff, printqdeg):
        tstr = ""
        if printqdeg == True:
            # Construct the target space with qdegs
            for t in self.pTargets:
                if self.pTargets.index(t) < len(self.pTargets)-1:
                    tstr = tstr + "R_{" + str(t) + "}\{" + str(diff.qdegr[t-1]) + "\}\oplus "
                else:
                    tstr = tstr + "R_{" + str(t) + "}\{" + str(diff.qdegr[t-1]) + "\}"
        else:
            # Construct the target space without qdegs
            for t in self.pTargets:
                if self.pTargets.index(t) < len(self.pTargets)-1:
                    tstr = tstr + "R_{" + str(t) + "}\oplus "
                else:
                    tstr = tstr + "R_{" + str(t) + "}"
        return tstr
    
    # Print it pretty(,) please!
    def printit(self, diff, printmatrix, printqdeg):
        hzm = self.haszeromap()
        sstr = self.sourcestring(diff, printqdeg)
        if hzm == True:
            pretty_print(LatexExpr(sstr))
        elif hzm == False:
            tstr = self.targetstring(diff, printqdeg)
            pretty_print(LatexExpr(self.sourcestring(diff, printqdeg) + "\overset{" + "}\longrightarrow "
                               + self.targetstring(diff, printqdeg)))
            if printmatrix == True:
                pretty_print(Matrix(R, self.pMatrix))

######################## Methods ########################


# A method that copies the passed Sage matrix and returns it as a new Sage matrix.
def copymatrix(A):
    matrix = []
    for i in range(A.nrows()):
        matrix.append(A.row(i))
    return Matrix(R, matrix)

# A global method to compare the number of zero entries of two matrices.
# Depending on the type of comparison (">" or ">="), returns True if the
# first argument has strictly more (or equal) zero entries than the second,
# and False otherwise.
def morezeros(matrix1, matrix2, comparison):
    s = 0
    t = 0
    for a in matrix1:
        if a == 0:
            s += 1
    for b in matrix2:
        if b == 0:
            t += 1
    if comparison == ">=":
        if s > t or s == t:
            return True
        else:
            return False
    else:
        if s > t:
            return True
        else:
            return False

# Prints a horizontal line
def printhline():
    pretty_print("--------------------------------------------------------------------------------")
    return

# A method for printig the matrix of the differentials. Takes as argument:
# bool printit (prints the matrices if True)
# bool pretty (print the matrices in LaTeX if True)
def printmatrices(printit, pretty):
    if printit == True:
        if pretty == True:
            for subcomp in subcomplexes:
                diffstoprint = subcomp.scDifferentials
                for A in diffstoprint:
                    print("Reduced matrix of d^{0}".format(A.hdeg) + ":")
                    pretty_print(A.matrix)
        else:
            for subcomp in subcomplexes:
                diffstoprint = subcomp.scDifferentials
                for A in diffstoprint:
                    print("Reduced matrix of d^{0}".format(A.hdeg) + ":")
                    print(A.matrix)
            # Count number of non-zero entries to see the
            # efficiency of the reduction.
            #z = 0
            #for l in A.matrix.list():
            #    if l != 0:
            #        z += 1
            #print('Number of non-zero entries: ', z)
    else:
        return

# A method for printing the elementary pieces of the differentials. Takes as arguments:
# bool printit (prints pieces if True)
# bool printmatrix (prints the matrix of pieces if True)
# bool printqdeg (prints quantum degrees if True)
def printpieces(printit, printmatrix, printqdeg):
    if printit == True:
        printhline()
        for subcomp in subcomplexes:
            diffstoprint = subcomp.scDifferentials
            for A in diffstoprint:
                print("\nElementary Pieces of d^{0}".format(A.hdeg) + ":")
                if A.epieces != []:
                    for p in A.epieces:
                        p.printit(A, printmatrix, printqdeg)
                else:
                    print("\nNone")
    else:
        return

# A method for printing the chains. Takes as arguments:
# bool printit (prints chains if True)
# bool printmatrix (prints matrices of the chain if True)
# bool printqdeg (prints quantum degrees if True)
# str sortby (sorts output, accepts "rank" or "length")
# str order (orders output ascending or descending, accepts "asc" or "desc")
# bool hidesmallchains (doesn't print chains of length <= 2 if True)
def printchains(printit, printmatrix, printqdeg, sortby, order, hidesmallchains):
    global chains
    if printit == True:
        printhline()
        if sortby == "rank":
            # Sort chains by rank in the specified order
            if order == "asc":
                csbyrank = sorted(chains, key=lambda chain: chain.cRank)
            elif order == "desc":
                csbyrank = list(reversed(sorted(chains, key=lambda chain: chain.cRank)))
            else:
                print("Error, specified unsupported order (need asc or desc)")
            # Truncate chains of length < 3 if hidesmallchains is True
            if hidesmallchains == True:
                csbyrank = [c for c in csbyrank if c.cLength >= 3]
                if csbyrank == []:
                    print("No summands of length >= 3 found!")
                    return
            # Print the chains
            rank = csbyrank[0].cRank
            print("Summands of rank " + str(rank) + ":")
            for i in range(len(csbyrank)):
                csbyrank[i].printit(printmatrix, printqdeg)
                if i+1 < len(csbyrank) and csbyrank[i+1].cRank != rank:
                    rank = csbyrank[i+1].cRank
                    print("Summands of rank " + str(rank) + ":")
        
        elif sortby == "length":
            # Sort chains by length in the specified order
            if order == "asc":
                csbylength = sorted(chains, key=lambda chain: chain.cLength)
            elif order == "desc":
                csbylength = list(reversed(sorted(chains, key=lambda chain: chain.cLength)))
            else:
                print("Error, specified unsupported order (need asc or desc)")
            # Truncate chains of length < 3 if hidesmallchains is True
            if hidesmallchains == True:
                csbylength = [c for c in csbylength if c.cLength >= 3]
                if csbylength == []:
                    print("No summands of length >= 3 found!")
                    return
            # Print the chains
            length = csbylength[0].cLength
            print("Summands of length " + str(length) + ":")
            for i in range(len(csbylength)):
                csbylength[i].printit(printmatrix, printqdeg)
                if i+1 < len(csbylength) and csbylength[i+1].cLength != length:
                    length = csbylength[i+1].cLength
                    print("Summands of length " + str(length) + ":")
        else:
            print("Error, specified unsupported sort key (need rank or length)")
    else:
        return

# A method for printing chains with maps containing G^n with n >= 2.
# (Work in progress)
def printhighpowerchains(printit, printmatrix, printqdeg):
    if printit == True:
        printhline()
        hpchains = []
        mstrng = []
        for c in chains:
            for p in c.cChain:
                mstring = [str(a) for t in p.pMatrix for a in t]
                for s in mstring:
                    if "^" in s:
                        hpchains.append(c)
                        break
        for c in hpchains:
            c.printit(printmatrix, printqdeg)
    else:
        return
                
# A method for printing the piece of odd rank. Takes as arguments:
# bool printit (prints the piece of odd rank if True)
# bool printmatrix (prints the matrices of the piece with odd rank if True)
# bool printqdeg (prints the quantum degrees if True)
def printoddpiece(printit, printmatrix, printqdeg):
    if printit == True:
        printhline()
        print("Piece of odd rank: ")
        if len(oddpiece) == 1:
            oddpiece[0].printit(printmatrix, printqdeg)
        elif len(oddpiece) > 1 and isinstance(oddpiece[0], Piece) == False:
            print("Error, more than one piece of odd rank found!")
        elif len(oddpiece) == 0:
            print("Error, no piece of odd rank found!")
    else:
        return

######################## __main__ ########################

# We work over R = Z[G]
R.<G> = PolynomialRing(ZZ, 1)

# Read the output file
with open(fname) as file:
    content = file.readlines()
    file.close()
    
######## Trim the content to your needs ########

content = [x.strip() for x in content] # Remove linebreaks
content.pop(0) # Remove Frobenius algebra
i = 0
while i <= len(content)-1:
    if content[i][0] in {"R","E"}:
        content.pop(i)
    else:
        i += 1
khopoly = content[len(content)-1] # Store the Khovanov Polynomial
khopoly = khopoly.replace(" ", "")
khopoly = khopoly.replace("^", "")
khopoly = khopoly.replace("+", "")
content.pop(len(content)-1) # Remove Khovanov polynomial
if "]," in content[len(content)-1]:
    content.pop(len(content)-1) # Remove PD code
for i in range(len(content)):
    if "X" in content[i]:
        content[i] = content[i].replace("X", "0")
    if "a" in content[i]:
        content[i] = content[i].replace("a", "0")
    if "b" in content[i]:
        content[i] = content[i].replace("b", "G")
    content[i] = content[i].replace(" ", "")
    content[i] = content[i].replace("[", "")
    content[i] = content[i].replace("]", "")
    #content[i] = content[i].replace("(", "")
    #content[i] = content[i].replace(")", "")

######## Initialize the subcomplexes ########

# Initialize matrices
subcomplexes = [Subcomplex()]
subcomplextracker = 0 # To track which subcomplex we are initializing
i = 0
while i < len(content):
    if content[i] == "":
        subcomplexes.append(Subcomplex()) # Found new subcomplex!
        subcomplextracker += 1
        nmbofnulls = 1 # Check how many [] are following to compute increment of i
        j = i+1 # At this point, i+1 should always be <len(content)!
        if j >= len(content):
            raise Exception("Parsing error, found isolated empty matrix []!")
        while content[j] == "":
            nmbofnulls += 1
            j += 1
            if j >= len(content):
                break
        #if i+nmbofnulls < len(content):
        i += nmbofnulls
        #else:
            #break
    else:
        mat = [] # Read out the matrix from the current line
        row = []
        entry = ""
        for j in range(len(content[i])):
            if j == len(content[i])-1:
                entry += content[i][j]
                row.append(R(entry))
                mat.append(row)
                row = []
                entry = ""
            elif content[i][j] == ";":
                row.append(R(entry))
                mat.append(row)
                row = []
                entry = ""
            elif content[i][j] == ",":
                row.append(R(entry))
                entry = ""
            else:
                entry += content[i][j]
        subcomplexes[subcomplextracker].scMatrices.append(Matrix(R, mat))
        i += 1

# Initialize hdegs and qdegs
hdegs = [] # Stores hdegs of current subcomplex
hdegstring =""
hdegtracker = "" # To check if a change of hdeg occured
hdegchange = False
qdegs = [] # Stores the qdegs of the current chain object
overallqdegs = [] # Stores the qdegs of all chain objects in the subcomplex
qdegstring = ""
i = 0
counter = 0 # To count how many chars to skip from "t" to the next "q"
subcomplextracker = 0
subcomplexchange = False
while i < len(khopoly):
    if khopoly[i] == "t":
        hdegstring = ""
        j = i+1
        if j >= len(khopoly):
            raise Exception("Error, unsupported Khovanov polynomial (doesn't end with a number)!")
            break
        while khopoly[j] != "q":
            hdegstring += khopoly[j]
            counter += 1
            j += 1
            if j >= len(khopoly):
                break
        if hdegtracker == "": # Treat first occurence of "t" here
            hdegs.append(int(hdegstring))
            hdegtracker = hdegstring
        elif hdegtracker != hdegstring:
            difference = int(hdegstring)-int(hdegtracker) # Prepare to switch subcomplex if difference > 1
            if difference > 1:
                if int(hdegtracker) == 0 and subcomplextracker != 0:
                    #tempqdegs = subcomplexes[subcomplextracker-1].scQdegs
                    #subcomplexes[subcomplextracker-1].scQdegs = []
                    subcomplexes.insert(subcomplextracker, Subcomplex())
                    #subcompleces[subcomplextracker-1].scQdegs = tempqdegs
                    #subcomplexes[subcomplextracker-1].scHdegs = [0]
                subcomplexes[subcomplextracker].scHdegs = hdegs
                subcomplexchange = True
                hdegs = []
            else:
                subcomplexchange = False
            hdegs.append(int(hdegstring))
            hdegtracker = hdegstring
            hdegchange = True
        else:
            hdegchange = False
        i += counter+1
        counter = 0
    if khopoly[i] == "q":
        j = i+1
        if j >= len(khopoly):
            raise Exception("Error, unsupported Khovanov polynomial (doesn't end with a number)!")
            break
        while khopoly[j] != "t":
            qdegstring += khopoly[j]
            counter += 1
            j += 1
            if j >= len(khopoly):
                break
        if hdegchange == False:
            qdegs.append(int(qdegstring))
            if i+counter+1 >= len(khopoly):
                overallqdegs.append(qdegs)
            qdegstring = ""
        elif hdegchange == True:
            overallqdegs.append(qdegs)
            if subcomplexchange == True:
                subcomplexes[subcomplextracker].scQdegs = overallqdegs
                overallqdegs = []
                subcomplextracker += 1 # Change subcomplex after qdeg, not hdeg!
            qdegs = []
            qdegs.append(int(qdegstring))
            if i+counter+1 >= len(khopoly):
                overallqdegs.append(qdegs)
            qdegstring = ""
            hdegstring = ""
        i += counter+1
        counter = 0
    else:
        raise Exception("Index error while parsing the Khovanov polynomial (need khopoly[i] == t or q)")

# Store hdegs and overallqdegs of the last subcomplex
subcomplexes[len(subcomplexes)-1].scHdegs = hdegs
subcomplexes[len(subcomplexes)-1].scQdegs = overallqdegs

# Initialize differentials
for subcomp in subcomplexes:
    # Special case if a chain object is isolated (should only happen for C^0)
    if subcomp.scMatrices == []:
        if len(subcomp.scHdegs) != 1 or len(subcomp.scQdegs) != 1 or subcomp.scHdegs[0] != 0:
            raise Exception("Subcomplex creation error, unsupported isolated chain object (only C^0 accepted)")
        subcomp.scDifferentials.append(Differential(subcomp.scHdegs[0], Matrix(R, [[0]]),
                                                    subcomp.scQdegs[0], [0]))
        continue
    # Check if everything is as it should be
    if len(subcomp.scMatrices)+1 != len(subcomp.scHdegs):
        raise Exception("Subcomplex creation error, number of matrices must equal number of hdegs + 1!")
    if len(subcomp.scHdegs) != len(subcomp.scQdegs):
        raise Exception("Subcomplex creation error, number of hdegs must equal number of qdegs!")
    # Initialize differentials
    for i in range(len(subcomp.scHdegs)-1):
        subcomp.scDifferentials.append(Differential(subcomp.scHdegs[i], subcomp.scMatrices[i],
                                    subcomp.scQdegs[i], subcomp.scQdegs[i+1]))
    #Implement nasty exception if C^0 is the end of the complex
    if subcomp.scHdegs[-1] == 0:
        subcomp.scMatrices.append(Matrix(R, [[0]*subcomp.scMatrices[-1].nrows()]))
        subcomp.scQdegs.append([0])
        subcomp.scDifferentials.append(Differential(subcomp.scHdegs[-1], subcomp.scMatrices[-1],
                                                   subcomp.scQdegs[-2], subcomp.scQdegs[-1]))

######## Check if C^0 is start of the chain complex ########

# If C^0 is the start of the complex, set pawnflag to True in order to
# indicate the correct method to search for the pawn after the matrix reduction.

pawnflag = False
for subcomp in subcomplexes:
    if subcomp.scDifferentials[0].hdeg == 0 and len(subcomp.scDifferentials) != 1:
        pawnflag = True

######## Matrix reduction ########

# The reduction algorithm runs twice through the list of differentials. In the first iteration, we
# reduce the matrices as much as possible. However, since the differentials are part of a chain complex,
# reducing a matrix might change the previous, already reduced matrix again. Because of this, we iterate
# a second time over the differentials but this time in reversed order and only perform column operations
# which change subsequent but not previous matrices.
isolated = 0
for subcomp in subcomplexes:
    
    # Check if there is an isolated subcomplex (should only be C^0) and append a pawn if it's the case.
    if subcomp.scMatrices == []:
        isolated += 1
        if isolated > 1:
            raise Exception("Error, more than one isolated subcomplex found (should only be C^0)!")
        if len(subcomp.scDifferentials) != 1:
            raise Exception("Error, no differential for isolated C^0 found!")
        subcomp.scDifferentials[0].epieces.append(Piece([1],[1],[[Map(1,1,0)]]))
        subcomp.scChains.append(Chain([subcomp.scDifferentials[0].epieces[0]],
                                                          [subcomp.scDifferentials[0]],
                                                          subcomp.scDifferentials[0].hdeg,
                                                          subcomp.scDifferentials[0].hdeg))
        continue
    
    diffs = subcomp.scDifferentials

    # First iteration
    for k in range(len(diffs)):
        # Keep track of the list index of the current object in the loop
        A = diffs[k].matrix

        # Set up variables for counting zero's before (zb) and after (za)
        # an iteration of the elimination process
        zb = 0
        za = 0
        for l in A.list():
            if l == 0:
                zb += 1

        # Main procedure, runs through each matrix entry twice searching first for row and then
        # for column operations that either increase the number of zero entries in the matrix or
        # keep them the same. Also performs the corresponding inverse operations on the previous
        # and next differentials. Stops as soon as no new zero entries can be created.
        done = False
        safetycheck = 0
        while not done:
            # Search for row operations that introduce more zeros
            for i in range(A.nrows()-1):
                for j in range(A.ncols()):
                    z = A[i,j]
                    for m in range(A.nrows()-1-i):
                        if z != 0 and A[m+i+1,j] != 0:
                            if (A[m+i+1,j] % z) == 0 and (z % A[m+i+1,j]) != 0:
                                B = copymatrix(A)
                                B.add_multiple_of_row(m+i+1,i,-(B[m+i+1,j]/z))
                                if morezeros(B.row(m+i+1), A.row(m+i+1), ">=") == True:
                                    # Inverse operation has to be done before making the operation on A!
                                    if k+1 < len(diffs):
                                        (diffs[k+1].matrix).add_multiple_of_column(i,m+i+1,A[m+i+1,j]/z)
                                    A.add_multiple_of_row(m+i+1,i,-(A[m+i+1,j]/z))
                                    continue
                            if (z % A[m+i+1,j]) == 0 and (A[m+i+1,j] % z) != 0:
                                B = copymatrix(A)
                                B.add_multiple_of_row(i,m+i+1,-(z/B[m+i+1,j]))
                                if morezeros(B.row(i), A.row(i), ">=") == True:
                                    if k+1 < len(diffs):
                                        diffs[k+1].matrix.add_multiple_of_column(m+i+1,i,z/A[m+i+1,j])
                                    A.add_multiple_of_row(i,m+i+1,-(z/A[m+i+1,j]))
                                    z = 0
                                    break
                            if (A[m+i+1,j] % z) == 0 and (z % A[m+i+1,j]) == 0:
                                B1 = copymatrix(A)
                                B2 = copymatrix(A)
                                B1.add_multiple_of_row(m+i+1,i,-(B1[m+i+1,j]/z))
                                B2.add_multiple_of_row(i,m+i+1,-(z/B2[m+i+1,j]))
                                checkB1 = list(B1.row(m+i+1))
                                checkB1.extend(list(B1.row(i)))
                                checkB2 = list(B2.row(m+i+1))
                                checkB2.extend(list(B2.row(i)))
                                if morezeros(checkB1, checkB2, ">="):
                                    if morezeros(B1.row(m+i+1), A.row(m+i+1), ">=") == True:
                                        # Inverse operation has to be done before making the operation on A!
                                        if k+1 < len(diffs):
                                            (diffs[k+1].matrix).add_multiple_of_column(i,m+i+1,A[m+i+1,j]/z)
                                        A.add_multiple_of_row(m+i+1,i,-(A[m+i+1,j]/z))
                                        continue
                                else:
                                    if morezeros(B2.row(i), A.row(i), ">=") == True:
                                        if k+1 < len(diffs):
                                            diffs[k+1].matrix.add_multiple_of_column(m+i+1,i,z/A[m+i+1,j])
                                        A.add_multiple_of_row(i,m+i+1,-(z/A[m+i+1,j]))
                                        z = 0
                                        break
            
            # Search for column operations that introduce more zeros
            for j in range(A.ncols()-1):
                for i in range(A.nrows()):
                    z = A[i,j]
                    for m in range(A.ncols()-1-j):
                        if z != 0 and A[i,m+j+1] != 0:
                            if (A[i,m+j+1] % z) == 0 and (z % A[i,m+j+1]) != 0:
                                B = copymatrix(A)
                                B.add_multiple_of_column(m+j+1,j,-(A[i,m+j+1]/z))
                                if morezeros(B.column(m+j+1), A.column(m+j+1), ">=") == True:
                                    if k-1 >= 0:
                                        diffs[k-1].matrix.add_multiple_of_row(j,m+j+1,A[i,m+j+1]/z)
                                    A.add_multiple_of_column(m+j+1,j,-(A[i,m+j+1]/z))
                                    continue
                            if (z % A[i,m+j+1]) == 0 and (A[i,m+j+1] % z) != 0:
                                B = copymatrix(A)
                                B.add_multiple_of_column(j,m+j+1,-(z/B[i,m+j+1]))
                                if morezeros(B.column(j), A.column(j), ">=") == True:
                                    if k-1 >= 0:
                                        diffs[k-1].matrix.add_multiple_of_row(m+j+1,j,z/A[i,m+j+1])
                                    A.add_multiple_of_column(j,m+j+1,-(z/A[i,m+j+1]))
                                    z = 0
                                    break
                            if (A[i,m+j+1] % z) == 0 and (z % A[i,m+j+1]) == 0:
                                B1 = copymatrix(A)
                                B2 = copymatrix(A)
                                B1.add_multiple_of_column(m+j+1,j,-(B1[i,m+j+1]/z))
                                B2.add_multiple_of_column(j,m+j+1,-(z/B2[i,m+j+1]))
                                checkB1 = list(B1.column(m+j+1))
                                checkB1.extend(list(B1.column(j)))
                                checkB2 = list(B2.column(m+j+1))
                                checkB2.extend(list(B2.column(j)))
                                if morezeros(checkB1, checkB2, ">=") == True:
                                    if morezeros(B1.column(m+j+1), A.column(m+j+1), ">=") == True:
                                        if k-1 >= 0:
                                            diffs[k-1].matrix.add_multiple_of_row(j,m+j+1,A[i,m+j+1]/z)
                                        A.add_multiple_of_column(m+j+1,j,-(A[i,m+j+1]/z))
                                        continue
                                else:
                                    if morezeros(B2.column(j), A.column(j), ">=") == True:
                                        if k-1 >= 0:
                                            diffs[k-1].matrix.add_multiple_of_row(m+j+1,j,z/A[i,m+j+1])
                                        A.add_multiple_of_column(j,m+j+1,-(z/A[i,m+j+1]))
                                        z = 0
                                        break
                                   
            # Count new total number of zeros in A
            for l in A.list():
                if l == 0:
                    za += 1
            if za > zb:
                zb = za
                za = 0
            else:
                # It can happen that no new zero entries are created, yet a column
                # operation enabled new row operations that increase the number of zeroes.
                # Because of this we iterate one more time after the first time no new
                # zero entries are created.
                if safetycheck == 0:
                    safetycheck = 1
                    zb = za
                    za = 0
                else:
                    done = True
    
    # Second iteration
    for k in reversed(range(len(diffs))):
        A = diffs[k].matrix
        zb = 0
        za = 0
        for l in A.list():
            if l == 0:
                zb += 1
        done = False
        while not done:
            """
            # Search for row operations that introduce more zeros but don't affect the number
            # of zeros of the previous differential
            for i in range(A.nrows()-1):
                for j in range(A.ncols()):
                    z = A[i,j]
                    for m in range(A.nrows()-1-i):
                        if z != 0 and A[m+i+1,j] != 0:
                            if (A[m+i+1,j] % z) == 0:
                                B = deepcopy(A)
                                B.add_multiple_of_row(m+i+1,i,-(B[m+i+1,j]/z))
                                if morezeros(B, A, ">") == True:
                                    # Inverse operation has to be done before making the operation on A!
                                    if k+1 < len(diffs):
                                        C = deepcopy(diffs[k+1].matrix)
                                        C.add_multiple_of_column(i,m+i+1,A[m+i+1,j]/z)
                                        if morezeros(C, diffs[k+1].matrix, ">=") == True:
                                            (diffs[k+1].matrix).add_multiple_of_column(i,m+i+1,A[m+i+1,j]/z)
                                            A.add_multiple_of_row(m+i+1,i,-(A[m+i+1,j]/z))
                                            continue
                            if (z % A[m+i+1,j]) == 0:
                                B = deepcopy(A)
                                B.add_multiple_of_row(i,m+i+1,-(z/B[m+i+1,j]))
                                if morezeros(B, A, ">") == True:
                                    if k+1 < len(diffs):
                                        C = deepcopy(diffs[k+1].matrix)
                                        C.add_multiple_of_column(m+i+1,i,z/A[m+i+1,j])
                                        if morezeros(C, diffs[k+1].matrix, ">=") == True:
                                            diffs[k+1].matrix.add_multiple_of_column(m+i+1,i,z/A[m+i+1,j])
                                            A.add_multiple_of_row(i,m+i+1,-(z/A[m+i+1,j]))
                                            z = 0
                                            break
                                            """
            # Search for column operations that introduce more zeros
            for j in range(A.ncols()-1):
                for i in range(A.nrows()):
                    z = A[i,j]
                    for m in range(A.ncols()-1-j):
                        if z != 0 and A[i,m+j+1] != 0:
                            if (A[i,m+j+1] % z) == 0:
                                B = deepcopy(A)
                                B.add_multiple_of_column(m+j+1,j,-(B[i,m+j+1]/z))
                                if morezeros(B.column(m+j+1), A.column(m+j+1), ">=") == True:
                                    if k-1 >= 0:
                                        diffs[k-1].matrix.add_multiple_of_row(j,m+j+1,A[i,m+j+1]/z)
                                    A.add_multiple_of_column(m+j+1,j,-(A[i,m+j+1]/z))
                                    continue
                            if (z % A[i,m+j+1]) == 0:
                                B = deepcopy(A)
                                B.add_multiple_of_column(j,m+j+1,-(z/B[i,m+j+1]))
                                if morezeros(B.column(j), A.column(j), ">=") == True:
                                    if k-1 >= 0:
                                        diffs[k-1].matrix.add_multiple_of_row(m+j+1,j,z/A[i,m+j+1])
                                    A.add_multiple_of_column(j,m+j+1,-(z/A[i,m+j+1]))
                                    z = 0
                                    break
            # Count new total number of zeros in A
            for l in A.list():
                if l == 0:
                    za += 1
            if za > zb:
                zb = za
                za = 0
            else:
                done = True

    ######## Search for Pawns ########

    # A pawn is found by checking if there is an integer n such that
    # the n-th row of d^i and the n-th column of d^i+1 are both zero.
    # If the pawnflag is True (i.e. C^0 is the start of the complex),
    # a pawn is given by the (unique) zero column of d^0.
    nomorepawns = False # Checker if there is more than one pawn
    if pawnflag == True:
        for j in range(diffs[0].matrix.ncols()):
            if diffs[0].matrix.column(j) == 0:
                if nomorepawns == False:
                    tempm = []
                    tempt = []
                    for k in range(diffs[0].matrix.nrows()):
                        tempm.append(Map(j+1,k+1,0))
                        tempt.append(k+1)
                    diffs[0].epieces.append(Piece([j+1],tempt,[tempm]))
                    subcomp.scChains.append(Chain([Piece([0],tempt,[tempm])],[diffs[0]],diffs[0].hdeg,diffs[0].hdeg))
                    nomorepawns == True
                elif nomorepawns == True:
                    raise Exception("Error, found more than one pawn!")
    else:
        for i in range(len(diffs)-1):
            for j in range(diffs[i].matrix.nrows()):
                r = diffs[i].matrix.row(j)
                c = diffs[i+1].matrix.column(j)
                if r == 0 and c == 0:
                    if nomorepawns == False:
                        tempm = []
                        tempt = []
                        for k in range(diffs[i+1].matrix.nrows()):
                            tempm.append(Map(j+1,k+1,0))
                            tempt.append(k+1)
                        diffs[i+1].epieces.append(Piece([j+1],tempt,[tempm]))
                        subcomp.scChains.append(Chain([Piece([j+1],tempt,[tempm])],[diffs[i+1]],diffs[i+1].hdeg,diffs[i+1].hdeg))
                        nomorepawns = True
                    elif nomorepawns == True:
                        raise Exception("Error, found more than one pawn!")
    
    ######## Elementary Pieces Construction Step I ########

    # Construct the pieces given by the columns of the matrix of a differential
    # (i.e. pieces with a single source and multiple targets). The pieces get
    # stored in Differential.temppieces.
    for A in diffs:
        ind = diffs.index(A)
        plist = []
        for j in range(A.matrix.ncols()):
            tempm = []
            tempt = []
            p = Piece([], [], [])
            for i in range(A.matrix.nrows()):
                if A.matrix[i,j] != 0:
                    tempt.append(i+1)
                    tempm.append(Map(j+1,i+1,A.matrix[i,j]))
            p.merge(Piece([j+1],tempt,[tempm]))
            if p.pTargets != []:
                plist.append(p)
        A.temppieces = plist

    ######## Elementary Pieces Construction Step II ########

    # Merge pieces from Step I that have common targets.
    # We iterate over (a copy of) the list Differential.temppieces from the
    # previous step and search for pieces with common targets. When two pieces, say
    # P and Q, have common targets, we merge P with Q and pop Q from the list of pieces.
    # We perform this procedure for a piece until no other piece with common targets can
    # be found. In the end, the list contains only elementary pieces and gets stored in
    # Differential.epieces.
    for A in diffs:
        temp = []
        pcopy = deepcopy(A.temppieces) # We don't want to pop pieces from the original list.
        i = 0
        len(pcopy)
        while i < len(pcopy):
            if pcopy[i].haszeromap() == True:
                raise Exception("There is a temppiece with a zero map!")
            j = i+1
            done = 0 # Indicates if there are no more pieces in pcopy that have common targets with pcopy[i]
            while j < len(pcopy):
                if pcopy[j].haszeromap() == True:
                    raise Exception("There is a temppiece with a zero map!")
                check = [a for a in pcopy[i].pTargets if a in pcopy[j].pTargets]
                if check != []:
                    temp.append(pcopy[j])
                    pcopy.pop(j)
                    done = 1
                else:
                    j += 1
            for q in temp:
                pcopy[i].merge(q)
            temp = []
            if done == 0:
                i += 1
        # Once the elementary pieces are created, construct the transformation matrix of each piece.
        for t in pcopy:
            t.constructmatrix()
        A.epieces.extend(pcopy)
    
    ######## Find chains ########

    # The procedure to find chains goes as follows: we iterate once over (a copy of) all
    # differentials, and at each step we look at the current and upcoming differential,
    # say d^k and d^{k+1}. We iterate over the elementary pieces of d^k and check for each piece p
    # if there are elementary pieces in d^{k+1} whose sources have non-empty intersection with
    # the targets of p. If several such pieces are found in d^{k+1}, they get merged. Once all such
    # pieces in d^{k+1} are found and merged, we have to check if there are sources that are not
    # contained in the targets of p. If this is the case, we have to extend the targets of p with
    # the corresponding sources. This causes a backwards reaction, because now we have to search
    # for elementary pieces in d^k that have common targets with p and merge them, which can introduce
    # new sources in p which affects the pieces in the chain of previous differentials. Because of this,
    # we restart the entire procedure to find chains when we have to extend the targets of p.
    # In the end, every differential contains exactly one piece of a chain that goes through this
    # differential, so all that remains to extract the chains is to run through the differentials
    # again and patch together pieces with common targets resp. sources. Note that next to the pieces,
    # a chain also stores the hdeg start and stop as well as the differentials it runs through.
    diffscopy = deepcopy(diffs)
    k = 0
    br = False
    while k < len(diffscopy)-1:
        A = diffscopy[k]
        B = diffscopy[k+1]
        i = 0
        while i < len(A.epieces):
            if A.epieces[i].haszeromap() == True:
                i += 1
                continue
            j = 0
            startpiece = 0
            startfound = False
            while j < len(B.epieces):
                if B.epieces[j].haszeromap() == True:
                    j += 1
                    continue
                check = [target for target in A.epieces[i].pTargets if target in B.epieces[j].pSources]
                if check != []:
                    if startfound == False:
                        startpiece = B.epieces[j]
                        startfound = True
                        j += 1
                        continue
                    else:
                        startpiece.merge(B.epieces[j])
                        B.epieces.pop(j)
                        continue
                else:
                    j += 1
            if startpiece != 0:
                if A.epieces[i].extendtargets(startpiece) == True:
                    A.epieces[i].searchcommontargets(diffscopy[k])
                    k = 0
                    br = True
                    break
            i += 1
        if br == True:
            br = False
            continue
        else:
            k += 1
    
    for k in range(len(diffscopy)):
        A = diffscopy[k]
        t = 0
        while t < len(A.epieces):
            if A.epieces[t].haszeromap() == True:
                t += 1
                continue
            tempchain = [A.epieces[t]]
            tracker = A.epieces[t]
            tempdifferentials = [diffs[k]]
            hdegstop = diffscopy[k].hdeg+1
            if k+1 < len(diffscopy):
                i = k+1
                while i < len(diffscopy):
                    j = 0
                    cont = False
                    done = False
                    while j < len(diffscopy[i].epieces):
                        if diffscopy[i].epieces[j].haszeromap() == True:
                            j += 1
                            continue
                        check = [target for target in tracker.pTargets if target in diffscopy[i].epieces[j].pSources]
                        if check != []:
                            if done == True:
                                raise Exception("Chain creation error!")
                            tempchain.append(diffscopy[i].epieces[j])
                            diffscopy[i].epieces.pop(j)
                            tempdifferentials.append(diffs[i])
                            hdegstop = diffscopy[i].hdeg+1
                            cont = True
                            done = True
                        else:
                            j += 1
                    if cont == True:
                        tracker = tempchain[-1]
                        i += 1
                    else:
                        break
            subcomp.scChains.append(Chain(tempchain,tempdifferentials,A.hdeg,hdegstop))
            t += 1

######## Find the piece of odd rank ########
oddpiece = []
for subcomp in subcomplexes:
    for c in subcomp.scChains:
        if c.cRank % 2 != 0:
            oddpiece.append(c)

#for c in oddpiece:
#    c.printit(False, False)
######## Print! ########
chains = []
for subcomp in subcomplexes:
    for c in subcomp.scChains:
        chains.append(c)

"""
for i in range(len(chains)):
    with open("piece{0}.txt".format(i), "w") as f:
        f.write("Frobenius algebra: Z[a, b][X] / (1*X^2 + b*X + a).\n")
        khopoly = ""
        for c in chains[i].cChain:
            matrix = str(c.pMatrix)
            matrix += "\n"
            matrix = matrix.replace("],", "];")
            f.write(matrix)
            for j in range(Matrix(R, c.pMatrix).ncols()):
                khopoly += "t^{hdeg}q^0".format(hdeg = chains[i].cStart+chains[i].cChain.index(c))
            if chains[i].cChain.index(c) == len(chains[i].cChain)-1:
                for j in range(Matrix(R, c.pMatrix).nrows()):
                    khopoly += "t^{hdeg}q^0".format(hdeg = chains[i].cStop)
        f.write("Reduced Homology:\n")
        f.write("Equivariant Homology:\n")
        f.write(khopoly)
    f.close
"""      
#printit, pretty
printmatrices(*prntmatrices)

#printit, printmatrix, printqdeg
printpieces(*prntpieces)

#printit, printmatrix, printqdeg, sortby (rank or length), order (asc or desc), hidesmallchains
printchains(*prntchains)

#printit, printmatrix, printqdeg
printhighpowerchains(False, True, True)

#printit, printmatrix, printqdeg
printoddpiece(*prntoddpiece)